# GIS with Python！！！
## Notebook "Weather Data" part
- Chomoku LLC: Hideyuki Ogawa
- My twitter([@ogawahideyuki](https://twitter.com/ogawahideyuki))
- Company twitter [@hijichomoku](https://twitter.com/hijichomoku)
- https://chomoku.com/

- Data: PTree
    - https://www.eorc.jaxa.jp/ptree/userguide_j.html
    - Observe the amount of the sunlight(2021/1/31)

In [2]:
!pip install netCDF4 &> /dev/null
!pip install rasterio &> /dev/null
!pip install --upgrade plotly &> /dev/null
!pip install pydeck &> /dev/null

In [3]:
from pathlib import Path
import xarray as xr
import pandas as pd
import plotly.figure_factory as ff
import plotly.express as px
import pydeck as pdk

In [5]:
# get file paths
# multiple net CDF files

p = Path('/content/drive/MyDrive/pyconjp2021/data/himawari/jaxa20210131')
cdfs = list(p.glob('*.nc'))
cdfs

[PosixPath('/content/drive/MyDrive/pyconjp2021/data/himawari/jaxa20210131/H08_20210130_2200_rFL010_FLDK.02701_02601.nc'),
 PosixPath('/content/drive/MyDrive/pyconjp2021/data/himawari/jaxa20210131/H08_20210130_2300_rFL010_FLDK.02701_02601.nc'),
 PosixPath('/content/drive/MyDrive/pyconjp2021/data/himawari/jaxa20210131/H08_20210131_0000_rFL010_FLDK.02701_02601.nc'),
 PosixPath('/content/drive/MyDrive/pyconjp2021/data/himawari/jaxa20210131/H08_20210131_0100_rFL010_FLDK.02701_02601.nc'),
 PosixPath('/content/drive/MyDrive/pyconjp2021/data/himawari/jaxa20210131/H08_20210131_0200_rFL010_FLDK.02701_02601.nc'),
 PosixPath('/content/drive/MyDrive/pyconjp2021/data/himawari/jaxa20210131/H08_20210131_0300_rFL010_FLDK.02701_02601.nc'),
 PosixPath('/content/drive/MyDrive/pyconjp2021/data/himawari/jaxa20210131/H08_20210131_0400_rFL010_FLDK.02701_02601.nc'),
 PosixPath('/content/drive/MyDrive/pyconjp2021/data/himawari/jaxa20210131/H08_20210131_0500_rFL010_FLDK.02701_02601.nc'),
 PosixPath('/content/dri

In [14]:
# read single file with xarray (fouth file)
da = xr.open_dataset(cdfs[3])
da

<xarray.Dataset>
Dimensions:              (band: 6, geometry: 17, latitude: 2601, longitude: 2701, time: 1)
Coordinates:
  * latitude             (latitude) float32 50.0 49.99 49.98 ... 24.01 24.0
  * longitude            (longitude) float32 123.0 123.0 123.0 ... 150.0 150.0
Dimensions without coordinates: band, geometry, time
Data variables:
    band_id              (band) int32 ...
    start_time           (time) datetime64[ns] ...
    end_time             (time) datetime64[ns] ...
    geometry_parameters  (geometry) float64 ...
    TAOT_02              (latitude, longitude) float32 ...
    TAAE                 (latitude, longitude) float32 ...
    PAR                  (latitude, longitude) float32 ...
    SWR                  (latitude, longitude) float32 ...
    UVA                  (latitude, longitude) float32 ...
    UVB                  (latitude, longitude) float32 ...
    QA_flag              (latitude, longitude) float32 ...
Attributes: (12/13)
    title:                          Himawari-8 AHI equal latitude-longitude m...
    id:                             H08_20210131_0100_rFL010_FLDK.02701_02601.nc
    date_created:                   2021-01-31T01:26:24Z
    pixel_number:                   2701
    line_number:                    2601
    upper_left_latitude:            50.0
    ...                             ...
    grid_interval:                  0.01
    band_number:                    6
    algorithm_version:              0100
    Ancillary meteorological data:  Unavailable
    Ancillary ozone data:           TOVS
    BRDF correction:                on (Morel and Maritorena 2001)

In [7]:
print(da.latitude.values)
print(da.start_time.values)

[50.   49.99 49.98 ... 24.02 24.01 24.  ]
['2021-01-31T01:00:53.709841408']


In [8]:
# data select
da.sel(latitude=slice(35.5, 35),
       longitude=slice(135, 135.5))['SWR']

<xarray.DataArray 'SWR' (latitude: 51, longitude: 51)>
array([[320.95   , 376.35   , 461.35   , ...,  48.9    ,  37.15   ,   4.65   ],
       [269.1    , 372.75   , 436.85   , ...,  47.65   ,  28.1    ,  13.     ],
       [367.9    , 384.55002, 395.05002, ...,  17.1    ,   4.45   ,  22.75   ],
       ...,
       [500.7    , 505.6    , 505.35   , ..., 500.55002, 493.05002, 501.75   ],
       [495.75   , 495.55002, 501.75   , ..., 389.30002, 471.65   , 523.35004],
       [489.2    , 489.05002, 496.45   , ..., 390.1    , 472.1    , 472.30002]],
      dtype=float32)
Coordinates:
  * latitude   (latitude) float32 35.5 35.49 35.48 35.47 ... 35.02 35.01 35.0
  * longitude  (longitude) float32 135.0 135.0 135.0 135.0 ... 135.5 135.5 135.5
Attributes:
    long_name:  Shortwave radiation
    units:      W/m^2
    valid_min:  0
    valid_max:  26000

In [9]:
# read all files
data = xr.open_mfdataset(cdfs, combine='nested', concat_dim='counts', parallel=True)
data

<xarray.Dataset>
Dimensions:              (band: 6, counts: 21, geometry: 17, latitude: 2601, longitude: 2701, time: 1)
Coordinates:
  * latitude             (latitude) float32 50.0 49.99 49.98 ... 24.01 24.0
  * longitude            (longitude) float32 123.0 123.0 123.0 ... 150.0 150.0
Dimensions without coordinates: band, counts, geometry, time
Data variables:
    band_id              (counts, band) int32 dask.array<chunksize=(1, 6), meta=np.ndarray>
    start_time           (counts, time) datetime64[ns] dask.array<chunksize=(1, 1), meta=np.ndarray>
    end_time             (counts, time) datetime64[ns] dask.array<chunksize=(1, 1), meta=np.ndarray>
    geometry_parameters  (counts, geometry) float64 dask.array<chunksize=(1, 17), meta=np.ndarray>
    TAOT_02              (counts, latitude, longitude) float32 dask.array<chunksize=(1, 2601, 2701), meta=np.ndarray>
    TAAE                 (counts, latitude, longitude) float32 dask.array<chunksize=(1, 2601, 2701), meta=np.ndarray>
    PAR                  (counts, latitude, longitude) float32 dask.array<chunksize=(1, 2601, 2701), meta=np.ndarray>
    SWR                  (counts, latitude, longitude) float32 dask.array<chunksize=(1, 2601, 2701), meta=np.ndarray>
    UVA                  (counts, latitude, longitude) float32 dask.array<chunksize=(1, 2601, 2701), meta=np.ndarray>
    UVB                  (counts, latitude, longitude) float32 dask.array<chunksize=(1, 2601, 2701), meta=np.ndarray>
    QA_flag              (counts, latitude, longitude) float32 dask.array<chunksize=(1, 2601, 2701), meta=np.ndarray>
Attributes: (12/13)
    title:                          Himawari-8 AHI equal latitude-longitude m...
    id:                             H08_20210130_2200_rFL010_FLDK.02701_02601.nc
    date_created:                   2021-01-30T22:24:58Z
    pixel_number:                   2701
    line_number:                    2601
    upper_left_latitude:            50.0
    ...                             ...
    grid_interval:                  0.01
    band_number:                    6
    algorithm_version:              0100
    Ancillary meteorological data:  JMA forcast
    Ancillary ozone data:           TOVS
    BRDF correction:                on (Morel and Maritorena 2001)

In [15]:
# select fourth file(easy)
print(data.isel(counts=3))
print(data.isel(counts=3).start_time.values)

<xarray.Dataset>
Dimensions:              (band: 6, geometry: 17, latitude: 2601, longitude: 2701, time: 1)
Coordinates:
  * latitude             (latitude) float32 50.0 49.99 49.98 ... 24.01 24.0
  * longitude            (longitude) float32 123.0 123.0 123.0 ... 150.0 150.0
Dimensions without coordinates: band, geometry, time
Data variables:
    band_id              (band) int32 dask.array<chunksize=(6,), meta=np.ndarray>
    start_time           (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    end_time             (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    geometry_parameters  (geometry) float64 dask.array<chunksize=(17,), meta=np.ndarray>
    TAOT_02              (latitude, longitude) float32 dask.array<chunksize=(2601, 2701), meta=np.ndarray>
    TAAE                 (latitude, longitude) float32 dask.array<chunksize=(2601, 2701), meta=np.ndarray>
    PAR                  (latitude, longitude) float32 dask.array<chunksize=(2601, 2701)

In [16]:
# convert to pandas (easy)

num_data = data.isel(counts=3)
start_time = num_data.start_time.values[0]
sel_data = num_data.sel(longitude=slice(135, 135.5), latitude=(35.5, 35)) # select area
sel_data = sel_data['SWR'].to_dataframe() # select channel and to dataframe
sel_data = sel_data.reset_index() 
sel_data = sel_data.rename({'counts': 'dt'}, axis=1)
sel_data['dt'] = start_time

In [18]:
sel_data.head(10)

,latitude,longitude,SWR,dt
0,35.5,135.000000,320.950012,2021-01-31 01:00:53.709841408
1,35.5,135.009995,376.350006,2021-01-31 01:00:53.709841408
2,35.5,135.020004,461.350006,2021-01-31 01:00:53.709841408
3,35.5,135.029999,437.200012,2021-01-31 01:00:53.709841408
4,35.5,135.039993,466.250000,2021-01-31 01:00:53.709841408
5,35.5,135.050003,466.000000,2021-01-31 01:00:53.709841408
6,35.5,135.059998,464.649994,2021-01-31 01:00:53.709841408
7,35.5,135.070007,473.149994,2021-01-31 01:00:53.709841408
8,35.5,135.080002,479.350006,2021-01-31 01:00:53.709841408
9,35.5,135.089996,395.250000,2021-01-31 01:00:53.709841408


In [19]:
def make_target_value(data, counts, min_lon=135, max_lon=135.5, min_lat=35, max_lat=35.5):
    num_data = data.isel(counts=counts)
    start_time = num_data.start_time.values[0]
    sel_data = num_data.sel(longitude=slice(min_lon, max_lon), latitude=slice(max_lat, min_lat))
    sel_data = sel_data['SWR'].to_dataframe()
    sel_data = sel_data.reset_index()
    sel_data = sel_data.rename({'counts': 'dt'}, axis=1)
    sel_data['dt'] = start_time
    return sel_data

In [21]:
# make selected area's data

all = pd.DataFrame()

for i in range(len(cdfs)):
    d = make_target_value(data, i)
    all = pd.concat([all, d])
all = all.reset_index(drop=True)
all = all.sort_values('dt')
all1 = all.copy()
all1['dt'] = all1['dt'].astype(str)

In [31]:
f = open('/content/drive/MyDrive/pyconjp2021/data/mapbox-token.txt')
mapbox_token = f.read()

In [32]:
px.set_mapbox_access_token(mapbox_token)

In [33]:
all1

,latitude,longitude,SWR,dt
0,35.500000,135.000000,0.000000,2021-01-30 22:00:53.901672448
1729,35.169998,135.460007,0.000000,2021-01-30 22:00:53.901672448
1730,35.169998,135.470001,0.000000,2021-01-30 22:00:53.901672448
1731,35.169998,135.479996,0.000000,2021-01-30 22:00:53.901672448
1732,35.169998,135.490005,0.000000,2021-01-30 22:00:53.901672448
...,...,...,...,...
26879,35.330002,135.020004,24.750000,2021-01-31 08:00:53.867340800
26880,35.330002,135.029999,24.650000,2021-01-31 08:00:53.867340800
26881,35.330002,135.039993,24.500000,2021-01-31 08:00:53.867340800
26863,35.340000,135.369995,19.400000,2021-01-31 08:00:53.867340800


In [38]:
# plot with plotly
fig = ff.create_hexbin_mapbox(
    all1, lat='latitude', lon='longitude', animation_frame='dt',
    color='SWR', nx_hexagon=20, #color_continuous_scale="Cividis",
)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [35]:
all_groupby = all.groupby(['latitude', 'longitude'], as_index=False).sum()
all_groupby['SWR'] = all_groupby['SWR'].astype(int)
all_groupby_piv = all_groupby.pivot(index='latitude', columns='longitude')
all_groupby_piv

SWR                        ...                                 
longitude 135.000000 135.009995 135.020004  ... 135.479996 135.490005 135.500000
latitude                                    ...                                 
35.000000       6371       6370       6709  ...       4954       5331       5332
35.010002       6516       6515       6596  ...       4936       5313       5695
35.020000       6767       6483       6479  ...       5529       5484       5404
35.029999       6636       6422       6079  ...       5833       5661       5596
35.040001       6633       6419       6076  ...       5980       6231       6501
35.049999       6084       5933       5494  ...       6097       6500       6727
35.060001       5949       5646       5306  ...       6396       6483       6802
35.070000       5831       5247       4580  ...       6392       6479       6795
35.080002       5157       4857       4439  ...       6350       6217       6278
35.090000       5156       5324       5277  ...       6174       6009       5816
35.099998       5310       5324       5272  ...       5793       5046       5291
35.110001       4820       5293       5802  ...       5585       5234       5297
35.119999       5164       5408       5902  ...       5581       5231       5294
35.130001       5631       5624       5906  ...       5675       5675       5380
35.139999       5651       5646       6040  ...       6092       6090       5481
35.150002       5646       5644       6041  ...       6021       5656       5655
35.160000       5604       5831       5831  ...       5488       5627       5627
35.169998       5181       5135       5135  ...       5485       5625       5658
35.180000       4519       4583       4776  ...       5469       5635       5494
35.190002       4497       4581       5317  ...       5657       5411       5542
35.200001       4499       4577       5317  ...       6039       5741       5455
35.209999       4773       4750       4828  ...       5723       5597       5056
35.220001       4781       4773       4577  ...       5722       5596       5055
35.230000       4547       4619       4793  ...       5223       5115       5116
35.240002       4189       4654       4730  ...       4986       5242       5233
35.250000       4186       4651       4727  ...       5133       5467       4924
35.260002       4212       4503       4374  ...       5091       5190       4898
35.270000       4299       4138       4224  ...       4971       5113       5079
35.279999       3815       3814       4394  ...       5185       5183       5154
35.290001       3794       3795       4121  ...       4355       4353       5451
35.299999       3792       4119       4073  ...       3787       5304       5305
35.310001       4249       4071       4070  ...       3253       4435       4435
35.320000       4256       4050       4677  ...       3811       3516       3571
35.330002       4586       4387       4571  ...       3811       3516       3571
35.340000       4342       4332       4527  ...       3587       3071       2849
35.349998       4730       4645       5151  ...       3028       2806       2940
35.360001       4737       4654       5149  ...       3747       2946       2409
35.369999       4578       4406       4968  ...       4022       3743       2966
35.380001       4200       4473       4869  ...       4046       3765       2982
35.389999       4550       4539       4705  ...       4436       4154       3489
35.400002       4639       4448       4662  ...       4697       4394       3966
35.410000       4637       4445       4662  ...       4573       4469       4094
35.419998       4204       4755       4546  ...       4643       4589       3837
35.430000       4570       4570       4521  ...       4618       4564       3814
35.440002       4419       4131       4133  ...       4518       4518       3935
35.450001       3873       3470       3470  ...       4013       4012       3811
35.459999       3872       3468       3668  ... 

In [37]:
# plot with pydeck

layer = pdk.Layer(
    "ColumnLayer", all_groupby, pickable=True, extruded=True, elevation_scale=1, get_position=['longitude', 'latitude'],
    get_elevation='SWR',
    get_fill_color=["SWR * 10", "SWR", "SWR * 10", 125],
    radius=1000,
)

view_state = pdk.ViewState(latitude=35.25, longitude=135.25, zoom=10, bearing=0, pitch=75)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "{longitude} / {latitude}: {SWR}"},)
r.to_html('column.html')

<IPython.core.display.Javascript object>

'<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n    <title>pydeck</title>\n        <script src="https://api.tiles.mapbox.com/mapbox-gl-js/v1.13.0/mapbox-gl.js"></script>\n        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css" />\n    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css" />\n    <script src=\'https://cdn.jsdelivr.net/npm/@deck.gl/jupyter-widget@~8.5.*/dist/index.js\'></script>\n    <style>\n    body {\n  margin: 0;\n  padding: 0;\n  overflow: hidden;\n}\n\n#deck-map-container {\n  width: 100%;\n  height: 100%;\n  background-color: black;\n}\n\n#map {\n  pointer-events: none;\n  height: 100%;\n  width: 100%;\n  position: absolute;\n  z-index: 1;\n}\n\n#deckgl-overlay {\n  z-index: 2;\n  background: none;\n}\n\n#deck-map-wrapper {\n  width: 100%;\n  height: 100%;\n}\n\n#deck-container {\n  width: 100v